In [1]:
import sys
sys.path.append('../hydroml')

from model import BaryonModel
from dataset import BaryonDataset
from IPython import display

from torch import nn

import torch

In [2]:
batch_size = 64
ngpu = 0
# Epochs set to '1' for testing
epochs = 2000
learning_rate = 1e-4
beta1 = 0.9
n_samples = 20000

In [3]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [4]:
dataset = BaryonDataset('../data/NetbaryonDis_OSG3DAuAu19.6_tune18.2_wBulk_22momdeltaf', n_samples )

data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [5]:
model = BaryonModel()

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [7]:
loss_func = nn.MSELoss()

In [8]:
loss_total = []

for epoch in range(epochs):
    for i, data in enumerate(data_loader):
        baryons = data[0]
        protons = data[1]

        optimizer.zero_grad()

        output = model(baryons.float())

        loss = loss_func(output, protons.float())

        loss.backward()
        optimizer.step()

        loss_total.append(loss.detach().numpy())

        if i % 128 == 0:
            display.clear_output(wait=True)

            # Uncommenting this will plot the loss and the generated output vs the actual output of the training sample.
            #plot.plot_telemetry(loss_total, output.detach().numpy(), protons)

            print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(dataset) // batch_size}], loss: {loss:.4f}")

Epoch [2000/2000], Batch [1/38], loss: 0.1777


In [10]:
torch.save(model, '../models/baryon_model_19gev.pt')